# Preparação da base de dados para treinar os classificadores

In [1]:
from src import (
    Folder,
    SegModel,
    SegmentedList,
    segmentation_fails,
    ds2df,
    save_results,
    File2df,
    Best_rf,
    Rf,
    MLP,
    SVM,
    XGBoost,
    KNN,
    GBM,
    List2Best,
    Save_Model_PKL
    )

from os.path import join
DATA = "Dados"
MODEL = "Models"

In [2]:
#adquindo as imagens e seus respectivos rotulos
#dataBasePath = join(DATA,"dados")
dataBasePath = join(DATA,"changed","rotacionados")
#dataBasePath = join(DATA,"changed","segmentation")
images, labels = Folder(dataBasePath)

## Caso as imagens não estejam segmentadas, podemos segmentalas

In [3]:

segmentation_model = SegModel(join(MODEL,"best.pt"))
process_images = SegmentedList(images,segmentation_model,is_resized=False) # Por padrão redimensionar garante menor taxa de falha
#process_images = SegmentedList(images,segmentation_model,is_resized=True)

WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 8.8ms
Speed: 3.1ms preprocess, 8.8ms inference, 417.3ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 (no detections), 9.1ms
Speed: 3.0ms preprocess, 9.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 8.9ms
Speed: 2.4ms preprocess, 8.9ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 9.1ms
Speed: 2.4ms preprocess, 9.1m

### com as imagens processadas, precisamos descartar as imagens que não foram segmentadas e salvar seu nome em um arquivo com as falhas

In [4]:
fails = segmentation_fails(process_images,labels)

#print(f"Quantidade: {len(fails)}\nFalhas:",fails)
#print("% = ",len(fails)/len(images))

#msg = "falhas-normais-"
#msg = "falhas-redimensionadas-"


### Podemos salvar as falhas em um arquivo de texto para vizualizar depois

In [ ]:
"""
anotation_fails= join("docs","Falhas","teste.txt")

with open(anotation_fails, "w") as arquivo:
        for fail in fails:
            arquivo.write(fail + "\n")
"""

### fazendo a filtragem das imagens válidas

In [5]:
valid_images = []
valid_labels = []

for i in range(0,len(process_images),1):
    if labels[i] not in fails:
        valid_images.append(process_images[i])
        valid_labels.append(labels[i])
        

#print(len(valid_labels),len(valid_images))

# Extraindo as Caracteristicas de interesse da base de dados

In [6]:
#df = ds2df(process_images,labels)
#df = ds2df(valid_images,valid_labels)
df = ds2df(images,labels)

### Visando evitar repetir o processamento para a mesma base de dados podemos exportar a mesma para um arquivo e apenas ler a partir do arquivo quando quisermos novamente

In [7]:

save_results(df,DATA,"Rotacionado",".csv")

### Carregando nossos dados a partir de um dataset .csv

In [2]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
df = File2df(join(DATA,"rotacionadas.csv"))

#df.drop("Unnamed: 0",axis=1,inplace=True)
#df.columns

.csv


# Treinando os classificadores

In [8]:
rf = Best_rf(df,join("docs","classification"))


In [9]:
Save_Model_PKL(rf,join(MODEL,"RF"))

### Vamos pegar a melhor versão de cada modelo

In [7]:
bests_list = []

funcs = [Rf, MLP,SVM,XGBoost,KNN,GBM]
names = ["RF","MLP","SVM","XGBoost","KNN","GBM"]

args = (0.1,0.2,0.3)

for i,func in enumerate(funcs,0):

    for arg in args:
        
        model, accuracy, precision, recall, f1, kappa = func(df,arg)
        bests_list.append((names[i],arg,accuracy, precision, recall, f1, kappa))
    
bests_list

[('RF', 0.1, 92.24, 90.12, 98.65, 94.19, 82.58),
 ('RF', 0.2, 90.52, 88.69, 98.03, 93.12, 77.97),
 ('RF', 0.3, 86.78, 86.5, 93.61, 89.91, 70.83),
 ('MLP', 0.1, 73.28, 74.71, 87.84, 80.75, 38.0),
 ('MLP', 0.2, 65.09, 85.86, 55.92, 67.73, 33.21),
 ('MLP', 0.3, 63.79, 84.96, 51.6, 64.2, 31.75),
 ('SVM', 0.1, 74.14, 73.91, 91.89, 81.93, 38.3),
 ('SVM', 0.2, 75.43, 76.54, 90.13, 82.78, 40.9),
 ('SVM', 0.3, 71.55, 74.19, 84.02, 78.8, 36.07),
 ('XGBoost', 0.1, 96.55, 96.05, 98.65, 97.33, 92.46),
 ('XGBoost', 0.2, 92.67, 91.41, 98.03, 94.6, 83.24),
 ('XGBoost', 0.3, 88.51, 88.74, 93.61, 91.11, 74.88),
 ('KNN', 0.1, 83.62, 89.86, 83.78, 86.71, 65.43),
 ('KNN', 0.2, 83.62, 89.04, 85.53, 87.25, 64.38),
 ('KNN', 0.3, 81.03, 86.26, 83.11, 84.65, 59.86),
 ('GBM', 0.1, 87.93, 85.71, 97.3, 91.14, 72.46),
 ('GBM', 0.2, 85.78, 86.06, 93.42, 89.59, 67.26),
 ('GBM', 0.3, 82.76, 82.45, 92.24, 87.07, 61.45)]

In [11]:
output = join("docs","classification")

In [12]:
#best = List2Best(bests_list,output,f"{len(valid_images)}.csv",True)
best = List2Best(bests_list,output,f"F{len(images)}.csv",True)
best

'XGBoost'

# caso gostemos do resultado e desejemos salvar o modelo

In [ ]:
#Save_model(model,join(MODEL,best))

# Avaliando o comparativo dos modelos 

In [ ]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
result_models = File2df(join(output,"not rf.csv"))

#result_models.drop("Unnamed: 0",axis=1,inplace=True)
result_models.columns

In [ ]:
result_models

In [ ]:
rotulos = result_models.Modelo.unique()
rotulos

In [ ]:
# Suponha que 'df' seja o DataFrame contendo os resultados dos modelos

# Rótulos dos modelos
model_labels = ["RF","MLP","SVM","XGBoost","KNN","GBM"]

# Lista para armazenar os melhores resultados de cada modelo
best_results = []

# Iterar sobre os rótulos dos modelos
for model_label in model_labels:
    # Filtrar o DataFrame para o modelo atual
    model_df = result_models[result_models['Modelo'] == model_label]
    
    # Encontrar a linha com o melhor resultado para o modelo atual
    best_row = model_df.loc[model_df['Total'].idxmax()]
    
    # Adicionar o resultado à lista de melhores resultados
    best_results.append(best_row)

import pandas as pd
# Criar um DataFrame com os melhores resultados
print(best_results)
best_results_df = pd.DataFrame(best_results)

# Exibir o DataFrame com os melhores resultados
print(best_results_df)
